# CLARISSA Tutorial: Reservoir Simulation Basics

This notebook demonstrates how CLARISSA translates natural language into simulation decks.

## 1. Darcy's Law - The Foundation

The fundamental equation governing fluid flow in porous media is **Darcy's Law**:

$$
q = -\frac{kA}{\mu} \frac{\Delta P}{L}
$$

Where:
- $q$ = volumetric flow rate (m³/s)
- $k$ = permeability (m²)
- $A$ = cross-sectional area (m²)
- $\mu$ = dynamic viscosity (Pa·s)
- $\Delta P$ = pressure difference (Pa)
- $L$ = length of the flow path (m)

## 2. System Architecture

```mermaid
flowchart LR
    subgraph UI["User Interface"]
        Voice["Voice"]
        Chat["Chat"]
        API["API"]
    end
    
    subgraph Core["CLARISSA Core"]
        NLP["NL Parser"]
        LLM["LLM Layer"]
        Val["Validator"]
    end
    
    subgraph Sim["Simulation"]
        Gen["Deck Generator"]
        OPM["OPM Flow"]
    end
    
    Voice --> NLP
    Chat --> NLP
    API --> NLP
    NLP --> LLM
    LLM --> Val
    Val --> Gen
    Gen --> OPM
```

## 3. Python Example: Calculating Pore Volume

In [ ]:
# Reservoir parameters
nx, ny, nz = 24, 25, 15  # Grid dimensions (SPE9 model)
dx, dy, dz = 300, 300, 50  # Cell sizes in feet
porosity = 0.087  # Average porosity
ntg = 1.0  # Net-to-gross ratio

# Calculate bulk volume
bulk_volume = nx * ny * nz * dx * dy * dz  # cubic feet

# Calculate pore volume
pore_volume = bulk_volume * porosity * ntg

# Convert to barrels (1 bbl = 5.615 ft³)
pore_volume_bbl = pore_volume / 5.615

print(f'Grid dimensions: {nx} x {ny} x {nz} = {nx*ny*nz:,} cells')
print(f'Pore volume: {pore_volume_bbl/1e6:.2f} MMbbl')

## 4. Material Balance Equation

The simplified material balance for an undersaturated reservoir:

$$
N_p B_o = N B_{oi} c_e \Delta P + W_e - W_p B_w
$$

Where the effective compressibility $c_e$ is:

$$
c_e = \frac{c_o S_o + c_w S_w + c_f}{1 - S_{wc}}
$$

In [ ]:
def calculate_ooip(np_cumulative, bo, boi, delta_p, ce):
    """Calculate Original Oil In Place using material balance."""
    N = (np_cumulative * bo) / (boi * ce * delta_p)
    return N

# Example calculation
N = calculate_ooip(
    np_cumulative=1_500_000,  # 1.5 MMSTB produced
    bo=1.25,
    boi=1.30,
    delta_p=500,  # psi
    ce=15e-6  # 1/psi
)

print(f'Estimated OOIP: {N/1e6:.1f} MMSTB')